In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

In [3]:
ch=pd.read_csv(r"C:\Users\Deven\Desktop\DATASET\Churn\Churn.csv")

In [5]:
ch.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
ch = ch.drop(columns=['customerID'])

In [9]:
ch.shape

(7043, 20)

In [12]:
column_names = ch.columns.tolist()
column_names

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [14]:
null_values = ch.isnull().sum()
null_values

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
ch[ch.select_dtypes(include='object').columns] = ch[ch.select_dtypes(include='object').columns].astype(str).apply(le.fit_transform)

In [21]:
ch.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,2505,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1466,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,157,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1400,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,925,1


In [24]:
from sklearn.model_selection import train_test_split
ch_train,ch_test=train_test_split(ch,test_size=0.2)

In [28]:
ch_train_X = ch_train.iloc[:, :-1]  # All columns except the last one
ch_train_Y = ch_train.iloc[:, -1]   # Only the last column (target variable)

# Separate features (X) and target (Y) for testing set
ch_test_X = ch_test.iloc[:, :-1]    # All columns except the last one
ch_test_Y = ch_test.iloc[:, -1]     # Only the last column (target variable)

In [30]:
ch_train_X.shape

(5634, 19)

In [32]:
ch_test_X.shape

(1409, 19)

In [34]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(ch_train_X,ch_train_Y)

RandomForestClassifier()

In [38]:
y_pred = rf.predict(ch_test_X)

In [49]:
Accuracy = accuracy_score(ch_test_Y, y_pred)
Accuracy

0.7892122072391767

In [52]:
print(classification_report(ch_test_Y, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86      1024
           1       0.66      0.46      0.55       385

    accuracy                           0.79      1409
   macro avg       0.74      0.69      0.70      1409
weighted avg       0.78      0.79      0.78      1409



In [54]:
print(confusion_matrix(ch_test_Y, y_pred))

[[933  91]
 [206 179]]


In [56]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
ch_train_X_resampled, ch_train_Y_resampled = smote.fit_resample(ch_train_X, ch_train_Y)

# Print the class distribution after oversampling
from collections import Counter
print("Before Oversampling:", Counter(ch_train_Y))
print("After Oversampling:", Counter(ch_train_Y_resampled))


Before Oversampling: Counter({0: 4150, 1: 1484})
After Oversampling: Counter({1: 4150, 0: 4150})


In [61]:
# Train the model on the oversampled data
rf1 = RandomForestClassifier(class_weight='balanced', random_state=42)
rf1.fit(ch_train_X, ch_train_Y)


# Make predictions on the test data
y_pred_ = rf.predict(ch_test_X)



In [63]:
accuracy = accuracy_score(ch_test_Y, y_pred_)
accuracy

0.7892122072391767

In [65]:
print(confusion_matrix(ch_test_Y, y_pred_))

[[933  91]
 [206 179]]


In [67]:
print(classification_report(ch_test_Y, y_pred_))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86      1024
           1       0.66      0.46      0.55       385

    accuracy                           0.79      1409
   macro avg       0.74      0.69      0.70      1409
weighted avg       0.78      0.79      0.78      1409



In [77]:
pip install xgboost --index-url=https://pypi.org/simple

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/70/58/2f94976df39470fb00eec2cb4f914dde44cd0df8d96483208bf7db4bc97e/xgboost-2.1.3-py3-none-win_amd64.whl.metadata (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x000001538A0B8A10>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))



In [76]:
# pip install xgboost

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/70/58/2f94976df39470fb00eec2cb4f914dde44cd0df8d96483208bf7db4bc97e/xgboost-2.1.3-py3-none-win_amd64.whl.metadata (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x0000022B97C5CF50>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))



In [72]:
from xgboost import XGBClassifier

# Train the XGBoost classifier
xgb = XGBClassifier(random_state=42)
xgb.fit(ch_train_X_resampled, ch_train_Y_resampled)

# Make predictions on the test data
y_pred__ = xgb.predict(ch_test_X)

ModuleNotFoundError: No module named 'xgboost'